# Summary
In this notebook we will setup the *VI-Ts triangle* model to run a single image using local meteorological data.


# TVDI setup
The main input data consists of any *GDAL* compatible raster image containing the radiometric temperature(s) and a vegetation index image. Optionally one can also include:

- A mask image which will filter the pixels to process/estimate the triangle edges (valid pixels==1).
- An image with the potential maximum Priestley-Taylor parameter for potential Evapotranspiration. Used only if the triangle will output the Evaporative Fraction (**EF**).

For more information about the inputs and outpus of pyTVDI click [here](./README_TVDI.md "README_TVDI.md")

## Select the input and output files and fill the information in all tabs 
You can press the *Load Configuration File* button to load a configuration text file which will upload all its information to the corresponding cells. 

You can also press the *Save Configuration File* button to save the configuration in a text file which can be used in future runs.

In [ ]:
from TVDI_IPython_Interface import TVDI_IPython_Interface # import the TVDI class object
setup=TVDI_IPython_Interface() # Create the setup instance from the PyTSEB class object
setup.TVDIWidget() # Load the TVDI configuration Widget

# Run TVDI
Once the `pyTVDI` is configured we parsed all the information from the widgets and ran the selected model (**TVDI** or **EF**). 

After the processing is done a GeoTiff file will be saved at the specified *Output Folder*:

- < Main Output Folder >/< Input File Basename >`_TVDI.tif`  

`pyTVDI` will also produce the following ancillary outputs:

- < Main Output Folder > `/line_equations_` < Input File Basename >`.txt`, with the computation parameters for the dry and wet edges.
- < Main Output Folder >`/`< Input File Basename > `_TVDI_cols_` < `col_ini` >`_`< `col_end` >`_rows_`< `row_ini` >`_`< `row_end` >`.png`, with the scatterplot of the Ts-VI triangle space and the estimated dry and wet edges.


# Display results
Now we can open the image and display the Ts/VI output:

In [ ]:
# Change to have a different colour stretch
high_flux=1 # Maximum flux value in the display
low_flux=0 # Minimum flux value in the display

from bokeh.plotting import *
from bokeh.palettes import RdYlBu11 as colortable
from bokeh.models.mappers import LinearColorMapper
from bokeh.io import output_notebook
import numpy as np
import gdal
from os.path import splitext,basename
output_notebook()
# Open the file
outputfile=setup.io_inf['output_dir']+'/'+splitext(basename(setup.io_inf['ndvi_file']))[0]+'_TVDI.tif'
fid=gdal.Open(outputfile,gdal.GA_ReadOnly)
rows=fid.RasterYSize
cols=fid.RasterXSize
# read each band and store the arrays
output=fid.GetRasterBand(1).ReadAsArray()
del fid
colortable=list(reversed(colortable))
map_output=LinearColorMapper(palette=colortable,high=high_flux,low=low_flux)

# Setup the figure
if setup.w_model.value==0:
    title='TVDI'
else:
    title='EF'
s1= figure(title=title, x_range=[0, cols], y_range=[0, rows])
s1.image(image=[np.flipud(output)],color_mapper=map_output,x=0,y=0)

# Add a colormap legend
y = np.linspace(low_flux,high_flux,len(colortable))
dy = y[1]-y[0]
ramp = figure(tools="", y_range = [0, 1], x_range = [low_flux,high_flux], plot_width = 650, plot_height=100)
ramp.toolbar_location=None
ramp.yaxis.visible = None
ramp.rect(x=y, y=0.5, color=colortable, width=dy, height = 1)

show(s1)
show(ramp);
